ここで用いられているソースコードは
https://github.com/Wildqat/Wildqat/blob/master/examples_ja/tutorial011_knapsack_with_integer_weights.ipynb
を参考にしています。

# ライブラリのimport 

In [1]:
import numpy as np
import pandas as pd
import wildqat as wq

# データの読み込み
データは
https://github.com/marushosummers/Saizeriya_1000yen/blob/master/sensai/saizeriya.db
をcsvに変換したものを使っています。

In [2]:
menu = pd.read_csv("./menu.csv", index_col=0)
menu.head(3)

,name,category,type,price,calorie,salt
id,,,,,,
1,彩りガーデンサラダ,sidedish,salad,299,130,1.1
2,小エビのサラダ,sidedish,salad,349,115,1.3
3,やわらかチキンのサラダ,sidedish,salad,299,134,1.2


# アイテムクラスの作成

In [3]:
class Item():
    def __init__(self, number, weight, cost):
        self.__number = number
        self.__name = name
        self.__weight = weight
        self.__cost = cost
    
    @property
    def name(self):
        return self.__name
    
    @property
    def weight(self):
        return self.__weight

    @property
    def cost(self):
        return self.__cost

    def __str__(self):
        return f"#{self.__number} (name : {self.name}, weight : {self.weight}, cost : {self.cost})"

# アイテムと重量制限の変数を定義

In [4]:
WEIGHT = 1000

items = []
for _id, _detail in menu.iterrows():
    number = _id
    name = _detail["name"]
    weight = _detail["price"]
    cost = _detail["calorie"]
    items.append(Item(number=number, weight=weight, cost=cost))

# ハミルトニアンの計算

途中の計算式で用いている$\rightarrow$はハミルトニアンから定数項を省いた場合や、ハミルトニアン全体を定数倍したときに用いています。

## パターン1 (最も有名なナップザック問題のハミルトニアン)

### $H_A$ について

$H_A$
$ \displaystyle
= \lambda \Big(1 - \sum_{n=1}^W y_n \Big)^2 + \mu \Big(\sum_{n=1}^W n y_n - \sum_{i=1}^N w_i x_i \Big)^2 
$
$ \displaystyle
= \lambda \Big[1 - 2 \sum_{n=1}^W y_n +  \Big(\sum_{n=1}^W y_n\Big)^2 \Big] + \mu \Big[\Big(\sum_{n=1}^W ny_n\Big)^2 -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_i x_i\Big)^2 \Big]
$
$ \displaystyle
= \lambda \Big[1 - 2 \sum_{n=1}^W y_n + \Big(\sum_{n=1}^W y_n^2 + \sum_{1 \leq n < m \leq W} 2 y_n y_m \Big) \Big]   + \mu\Big[\Big(\sum_{n=1}^W n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 n m y_n y_m\Big)  -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2w_i x_i w_j x_j \Big) \Big] 
$
$ \displaystyle
\rightarrow \sum_{n=1}^W (\mu n^2 - \lambda) y_n^2 + \sum_{1 \leq n < m \leq W} 2(\lambda + \mu nm) y_n y_m   + \Big(\sum_{n=1}^W \sum_{i=1}^N (-2) \mu n w_i y_n x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i w_j x_i x_j \Big)
$

## $H_B$ について
$H_B$
$ \displaystyle
= - \sigma \sum_{i=1}^{N}c_{i} x_{i}
$
$ \displaystyle
= - \sigma \sum_{i=1}^{N}c_{i} x_{i}^2
$

## $H_A + H_B$

$
H = H_A + H_B
$

$ \displaystyle
= \sum_{n=1}^W (\mu n^2 - \lambda) y_n^2 + \sum_{1 \leq n < m \leq W} 2(\lambda + \mu nm) y_n y_m  + \Big(\sum_{n=1}^W \sum_{i=1}^N (-2) \mu n w_i y_n x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i w_j x_i x_j \Big)  - \sigma \sum_{i=1}^{N}c_{i} x_{i}
$
$ \displaystyle
=  \sum_{n=1}^W (\mu n^2 - \lambda) y_n^2 + \sum_{1 \leq n < m \leq W} 2(\lambda + \mu nm) y_n y_m  + \Big(\sum_{n=1}^W \sum_{i=1}^N (-2) \mu n w_i y_n x_i\Big) +  \Big(\sum_{i=1}^N  (\mu w_{i}^2 - \sigma c_{i}) x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i w_j x_i x_j \Big) 
$

をプログラムで書く。

In [5]:
def get_qubo(items, wlimit, lamda, mu, sigma):
    # qubo行列を作成
    x_size = len(items)
    y_size = wlimit
    size = x_size + y_size
    
    # QUBO行列のx*x成分
    x_qubo = np.zeros((x_size, x_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        ci = items[i].cost
        for j in range(i, x_size):
            wj = items[j].weight
            if i == j:
                x_qubo[i][i] = mu * wi*wi - sigma * ci
            else:
                x_qubo[i][j] = 2*mu*wi*wj
    
    # QUBO行列のy*y成分
    y_qubo = np.zeros((y_size, y_size), dtype=int)
    for i in range(y_size):
        for j in range(i, y_size):
            if i == j:
                y_qubo[i][i] = mu*(i+1)*(i+1) - lamda
            else:
                y_qubo[i][j] = 2*(lamda + mu*(i+1)*(j+1))
                
    # QUBO行列のx*y成分
    xy_qubo = np.zeros((x_size, y_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        for j in range(y_size):
            xy_qubo[i][j] = -2*mu*(j+1)*wi
    
    # QUBO行列のy*x成分
    yx_qubo = np.zeros((y_size, x_size), dtype=int)
    
    # QUBO行列を接合する
    upper_qubo = np.concatenate((x_qubo, xy_qubo), axis=1)
    lower_qubo = np.concatenate((yx_qubo, y_qubo), axis=1)
    qubo = np.concatenate((upper_qubo, lower_qubo), axis=0)
    
    return qubo


## パターン2 (変数を減らし単純化してみた)

$H_A$
$ \displaystyle
= \mu \Big(W - \sum_{i=1}^N w_i x_i \Big)^2 
$

$ \displaystyle
= \mu \Big[W^2 -2W\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_i x_i\Big)^2 \Big]
$

$ \displaystyle
= \mu \Big[W^2  - 2W\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2w_i x_i w_j x_j \Big) \Big] 
$

$ \displaystyle
= \mu \Big[W^2 + \Big(\sum_{i=1}^N (- 2W) w_i x_i^2\Big) +  \Big(\sum_{i=1}^N w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2w_i x_i w_j x_j \Big) \Big] 
$

$ \displaystyle
\rightarrow \Big(\sum_{i=1}^N \mu (w_{i}^2 - 2 W w_{i}) x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i w_j x_i x_j \Big)
$

$H_B$はパターン１と同じものを用います。

In [6]:
def get_qubo2(items, wlimit, mu, sigma):
    # qubo行列を作成
    x_size = len(items)
    
    # QUBO行列のx*x成分
    x_qubo = np.zeros((x_size, x_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        ci = items[i].cost
        for j in range(i, x_size):
            wj = items[j].weight
            if i == j:
                x_qubo[i][i] = mu * ( wi*wi - 2 * wlimit * wi) - sigma * ci
            else: 
                x_qubo[i][j] = 2*mu*wi*wj
    return x_qubo

## パターン3 (アイテム重量の変数の項をアレンジ)

$H_A$
$ \displaystyle
= \lambda \Big(W - \sum_{n=1}^W n y_n \Big)^2 + \mu \Big(\sum_{n=1}^W n y_n - \sum_{i=1}^N w_i x_i \Big)^2 
$
$ \displaystyle
= \lambda \Big[W^2 - 2 W \sum_{n=1}^W n y_n +  \Big(\sum_{n=1}^W n y_n\Big)^2 \Big] + \mu \Big[\Big(\sum_{n=1}^W ny_n\Big)^2 -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_i x_i\Big)^2 \Big]
$
$ \displaystyle
= \lambda \Big[W^2 - 2W \sum_{n=1}^W n y_n + \Big(\sum_{n=1}^W n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 n m y_n y_m\Big) \Big]   + \mu\Big[\Big(\sum_{n=1}^W n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 n m y_n y_m\Big)  -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2w_i x_i w_j x_j \Big) \Big] 
$
$ \displaystyle
= \Big[\lambda W^2 - 2\lambda W \sum_{n=1}^W n y_n + \Big(\sum_{n=1}^W \lambda n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 \lambda n m y_n y_m\Big) \Big] + \Big[\Big(\sum_{n=1}^W \mu n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 \mu n m y_n y_m\Big)  -2 \mu \Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i x_i w_j x_j \Big) \Big] 
$
$ \displaystyle
\rightarrow  \Big[\sum_{n=1}^W (- 2) \lambda W n y_n + \Big(\sum_{n=1}^W \lambda n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 \lambda n m y_n y_m\Big) \Big] + \Big[\Big(\sum_{n=1}^W \mu n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 \mu n m y_n y_m\Big) + \Big(\sum_{n=1}^W \sum_{i=1}^N (-2) \mu n  w_i y_n x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i x_i w_j x_j \Big) \Big] 
$
$ \displaystyle
=  \Big[\sum_{n=1}^W \{(- 2) \lambda W n + \lambda n^2 + \mu n^2 \} y_n^2 + \Big(\sum_{1 \leq n < m \leq W} 2 (\lambda + \mu)n m y_n y_m\Big) \Big] + \Big[\Big(\sum_{n=1}^W \sum_{i=1}^N (-2) \mu n  w_i y_n x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i w_j x_i x_j \Big) \Big] 
$

$H_B$はパターン１と同じものを用います。

In [7]:
def get_qubo3(items, wlimit, lamda, mu, sigma):
    # qubo行列を作成
    x_size = len(items)
    y_size = wlimit
    size = x_size + y_size
    
    # QUBO行列のx*x成分
    x_qubo = np.zeros((x_size, x_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        ci = items[i].cost
        for j in range(i, x_size):
            wj = items[j].weight
            if i == j:
                x_qubo[i][i] = mu * wi*wi - sigma * ci
            else:
                x_qubo[i][j] = 2*mu*wi*wj
    
    # QUBO行列のy*y成分
    y_qubo = np.zeros((y_size, y_size), dtype=int)
    for i in range(y_size):
        for j in range(i, y_size):
            if i == j:
                y_qubo[i][i] = -2*lamda*wlimit*(i+1) + lamda*(i+1)**2 + mu*(i+1)**2
            else:
                y_qubo[i][j] = 2*(lamda+mu)*(i+1)*(j+1)
                
    # QUBO行列のx*y成分
    xy_qubo = np.zeros((x_size, y_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        for j in range(y_size):
            xy_qubo[i][j] = -2*mu*(j+1)*wi
    
    # QUBO行列のy*x成分
    yx_qubo = np.zeros((y_size, x_size), dtype=int)
    
    # QUBO行列を接合する
    upper_qubo = np.concatenate((x_qubo, xy_qubo), axis=1)
    lower_qubo = np.concatenate((yx_qubo, y_qubo), axis=1)
    qubo = np.concatenate((upper_qubo, lower_qubo), axis=0)
    
    return qubo

## パターン4 (パターン3に変数$y$の制約項を追加)

$H_A$
$ \displaystyle
= \omega \Big(1 - \sum_{n=1}^W y_n \Big)^2 + \lambda \Big(W - \sum_{n=1}^W n y_n \Big)^2 + \mu \Big(\sum_{n=1}^W n y_n - \sum_{i=1}^N w_i x_i \Big)^2 
$
$ \displaystyle
= \omega\Big[1 - 2 \sum_{n=1}^W y_n +  \Big(\sum_{n=1}^W y_n\Big)^2 \Big] + \lambda \Big[W^2 - 2 W \sum_{n=1}^W n y_n +  \Big(\sum_{n=1}^W n y_n\Big)^2 \Big] + \mu \Big[\Big(\sum_{n=1}^W ny_n\Big)^2 -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_i x_i\Big)^2 \Big]
$
$ \displaystyle
= \omega\Big[1 - 2 \sum_{n=1}^W y_n + \Big(\sum_{n=1}^W y_n^2 + \sum_{1 \leq n < m \leq W} 2 y_n y_m \Big) \Big] + \lambda \Big[W^2 - 2W \sum_{n=1}^W n y_n + \Big(\sum_{n=1}^W n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 n m y_n y_m\Big) \Big]   + \mu\Big[\Big(\sum_{n=1}^W n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 n m y_n y_m\Big)  -2\Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2w_i x_i w_j x_j \Big) \Big] 
$
$ \displaystyle
= \Big[\omega - 2 \omega \sum_{n=1}^W y_n + \Big(\sum_{n=1}^W \omega y_n^2 + \sum_{1 \leq n < m \leq W} 2 \omega y_n y_m \Big) \Big] + \Big[\lambda W^2 - 2\lambda W \sum_{n=1}^W n y_n + \Big(\sum_{n=1}^W \lambda n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 \lambda n m y_n y_m\Big) \Big] + \Big[\Big(\sum_{n=1}^W \mu n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 \mu n m y_n y_m\Big)  -2 \mu \Big(\sum_{n=1}^W ny_n\Big)\Big(\sum_{i=1}^N w_i x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i x_i w_j x_j \Big) \Big] 
$
$ \displaystyle
\rightarrow \Big[\sum_{n=1}^W (- 2) \omega y_n + \Big(\sum_{n=1}^W \omega y_n^2 + \sum_{1 \leq n < m \leq W} 2 \omega y_n y_m \Big) \Big] + \Big[\sum_{n=1}^W (- 2) \lambda W n y_n + \Big(\sum_{n=1}^W \lambda n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 \lambda n m y_n y_m\Big) \Big] + \Big[\Big(\sum_{n=1}^W \mu n^2 y_n^2 + \sum_{1 \leq n < m \leq W} 2 \mu n m y_n y_m\Big) + \Big(\sum_{n=1}^W \sum_{i=1}^N (-2) \mu n  w_i y_n x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i x_i w_j x_j \Big) \Big] 
$
$ \displaystyle
=  \Big[\sum_{n=1}^W \{(- 2) \lambda W n + \lambda n^2 + \mu n^2 - \omega \} y_n^2 + \Big(\sum_{1 \leq n < m \leq W} 2 ((\lambda + \mu)n m + \omega)y_n y_m\Big) \Big] + \Big[\Big(\sum_{n=1}^W \sum_{i=1}^N (-2) \mu n  w_i y_n x_i\Big) +  \Big(\sum_{i=1}^N \mu w_{i}^2 x_{i}^2 + \sum_{1 \leq i<j \leq N} 2 \mu w_i w_j x_i x_j \Big) \Big] 
$

$H_B$はパターン１と同じものを用います。

In [8]:
def get_qubo4(items, wlimit, omega, lamda, mu, sigma):
    # qubo行列を作成
    x_size = len(items)
    y_size = wlimit
    size = x_size + y_size
    
    # QUBO行列のx*x成分
    x_qubo = np.zeros((x_size, x_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        ci = items[i].cost
        for j in range(i, x_size):
            wj = items[j].weight
            if i == j:
                x_qubo[i][i] = mu * wi*wi - sigma * ci
            else:
                x_qubo[i][j] = 2*mu*wi*wj
    
    # QUBO行列のy*y成分
    y_qubo = np.zeros((y_size, y_size), dtype=int)
    for i in range(y_size):
        for j in range(i, y_size):
            if i == j:
                y_qubo[i][i] = -2*lamda*wlimit*(i+1) + lamda*(i+1)**2 + mu*(i+1)**2 - omega
            else:
                y_qubo[i][j] = 2*((lamda+mu)*(i+1)*(j+1)+omega)
                
    # QUBO行列のx*y成分
    xy_qubo = np.zeros((x_size, y_size), dtype=int)
    for i in range(x_size):
        wi = items[i].weight
        for j in range(y_size):
            xy_qubo[i][j] = -2*mu*(j+1)*wi
    
    # QUBO行列のy*x成分
    yx_qubo = np.zeros((y_size, x_size), dtype=int)
    
    # QUBO行列を接合する
    upper_qubo = np.concatenate((x_qubo, xy_qubo), axis=1)
    lower_qubo = np.concatenate((yx_qubo, y_qubo), axis=1)
    qubo = np.concatenate((upper_qubo, lower_qubo), axis=0)
    
    return qubo

# 計算を実行し結果を表示する関数

In [9]:
def run(annealer, times):
    for count in range(1, times+1):
        q = annealer.sa()

        answers = []
        weight = 0 
        cost = 0
        for i in range(len(items)):
            if q[i] == 1:
                answers.append(items[i])
                weight += items[i].weight
                cost += items[i].cost

        # 計算過程を可視化する
        # annealer.plot()

        print(f"count : {count}")
        print("y=1のindexは {}番目.".format(list(np.nonzero(q[len(items):])[0])))
        for answer in answers:
            print(f"selected : {answer}")
        print(f"total price : {weight}")
        print(f"total kcal : {cost}")
        print("")

# アニーラーの初期化と設定そして実行

selected : は選ばれたメニューを表しています。weightが値段でcostがカロリー数を示しています。<br>
total price : 総価格です<br>
total kcal : 総カロリー数です。<br>

## パターン１

In [10]:
annealer = wq.opt()
annealer.qubo = get_qubo(items, wlimit=WEIGHT, lamda=100000000, mu=1, sigma=500)
#: Descreasing rate of temperature [SA]
annealer.R = 0.95
#: Iterations [SA]
annealer.ite = 1000
run(annealer, 5)

count : 1
y=1のindexは [326]番目.
selected : #25 (name : ポテトのグリル, weight : 199, cost : 366)
selected : #28 (name : フォッカチオ, weight : 119, cost : 214)
selected : #82 (name : ミラノ風ドリア, weight : 299, cost : 542)
selected : #102 (name : ラージライス, weight : 219, cost : 454)
total price : 836
total kcal : 1576

count : 2
y=1のindexは [478]番目.
selected : #28 (name : フォッカチオ, weight : 119, cost : 214)
selected : #82 (name : ミラノ風ドリア, weight : 299, cost : 542)
selected : #86 (name : エビとイカのドリア, weight : 499, cost : 624)
total price : 917
total kcal : 1380

count : 3
y=1のindexは [623]番目.
selected : #28 (name : フォッカチオ, weight : 119, cost : 214)
selected : #58 (name : ほうれん草のグラタン(季節限定), weight : 399, cost : 521)
selected : #82 (name : ミラノ風ドリア, weight : 299, cost : 542)
selected : #102 (name : ラージライス, weight : 219, cost : 454)
total price : 1036
total kcal : 1731

count : 4
y=1のindexは [372]番目.
selected : #28 (name : フォッカチオ, weight : 119, cost : 214)
selected : #63 (name : アーリオ・オーリオ, weight : 299, cost : 560)
selec

多くの論文で紹介されているハミルトニアンを用いて計算を行った。まずはじめに$\lambda$を小さい値で設定した場合、total priceが大きくなり、選択されるメニューが大量になるという現象が起きたので、それを抑えるために極端に大きい値を係数に設定した。

結果として、total priceが$y$に依存しているような振る舞いをしている。$y=1$である要素が小さいとき(例えば$y=1$が 10番目/1000要素 にあり、他が$y=0$のとき)、total priceの値も小さい値になる傾向があるように見える。逆も然り。

個人的に試行錯誤をしたが結果的に↑↑の結果で落ち着いた。

## パターン２

In [11]:
annealer = wq.opt()
annealer.qubo = get_qubo2(items, wlimit=500, mu=1, sigma=500)
#: Descreasing rate of temperature [SA]
annealer.R = 0.95
#: Iterations [SA]
annealer.ite = 1000
run(annealer, 5)

count : 1
y=1のindexは []番目.
selected : #25 (name : ポテトのグリル, weight : 199, cost : 366)
selected : #28 (name : フォッカチオ, weight : 119, cost : 214)
selected : #63 (name : アーリオ・オーリオ, weight : 299, cost : 560)
selected : #101 (name : ライス, weight : 169, cost : 303)
selected : #102 (name : ラージライス, weight : 219, cost : 454)
total price : 1005
total kcal : 1897

count : 2
y=1のindexは []番目.
selected : #53 (name : パンチェッタのピザ, weight : 399, cost : 646)
selected : #63 (name : アーリオ・オーリオ, weight : 299, cost : 560)
selected : #102 (name : ラージライス, weight : 219, cost : 454)
total price : 917
total kcal : 1660

count : 3
y=1のindexは []番目.
selected : #25 (name : ポテトのグリル, weight : 199, cost : 366)
selected : #54 (name : 野菜ときのこのピザ, weight : 399, cost : 610)
selected : #61 (name : ミートソースボロニア風, weight : 399, cost : 582)
total price : 997
total kcal : 1558

count : 4
y=1のindexは []番目.
selected : #25 (name : ポテトのグリル, weight : 199, cost : 366)
selected : #28 (name : フォッカチオ, weight : 119, cost : 214)
selected : #82 (nam

変数が多いことが計算が不安定になる原因なのではという仮説を立てて、変数$x$のみで構成されるハミルトニアンを定式化した。計算結果は制約(total priceが1000以下)を安定して満たしており、理想的な計算結果となった。

しかし、パラメーターで重さを500と設定しており、本来の1000としなければならないことから、本来意図していなかったパラメータで結果的にうまく計算ができたというのが、正直なところです。

## パターン３

In [12]:
annealer = wq.opt()
annealer.qubo = get_qubo3(items, wlimit=900, lamda=1000000, mu=10, sigma=1)
#: Descreasing rate of temperature [SA]
annealer.R = 0.95
#: Iterations [SA]
annealer.ite = 1000
run(annealer, 5)

count : 1
y=1のindexは [32, 40, 96, 249, 480]番目.
selected : #16 (name : アスパラガスのオーブン焼き(季節限定), weight : 299, cost : 221)
selected : #63 (name : アーリオ・オーリオ, weight : 299, cost : 560)
selected : #109 (name : ミルクアイスのせシナモンフォッカチオ, weight : 319, cost : 346)
total price : 917
total kcal : 1127

count : 2
y=1のindexは [188, 756]番目.
selected : #78 (name : イカの墨入りスパゲッティ(Wサイズ), weight : 976, cost : 1220)
total price : 976
total kcal : 1220

count : 3
y=1のindexは [50, 225, 636]番目.
selected : #25 (name : ポテトのグリル, weight : 199, cost : 366)
selected : #26 (name : セロリのピクルス(季節限定), weight : 199, cost : 52)
selected : #56 (name : バッファローモッツァレラのピザ, weight : 499, cost : 575)
total price : 897
total kcal : 993

count : 4
y=1のindexは [79, 852]番目.
selected : #33 (name : デカンタ・500ml, weight : 399, cost : 0)
selected : #90 (name : 半熟卵のハヤシ＆ターメリックライス, weight : 568, cost : 728)
total price : 967
total kcal : 728

count : 5
y=1のindexは [190, 792]番目.
selected : #8 (name : コーンクリームスープ, weight : 149, cost : 142)
selected : #74 (nam

パターン３ではパタメータの数が少ないからか、total priceに重みをつけるとtotal kcalが上がらず、total kcalに重みをつけるとtotal priceにブレが生まれてしまうといった問題が生じました。

## パターン４

In [13]:
annealer = wq.opt()
annealer.qubo = get_qubo4(items, wlimit=900, omega=100000000, lamda=200, mu=1, sigma=500)
#: Descreasing rate of temperature [SA]
annealer.R = 0.95
#: Iterations [SA]
annealer.ite = 1000
run(annealer, 5)

count : 1
y=1のindexは [365]番目.
selected : #91 (name : ミックスグリル, weight : 599, cost : 823)
selected : #102 (name : ラージライス, weight : 219, cost : 454)
total price : 818
total kcal : 1277

count : 2
y=1のindexは [636]番目.
selected : #28 (name : フォッカチオ, weight : 119, cost : 214)
selected : #91 (name : ミックスグリル, weight : 599, cost : 823)
selected : #101 (name : ライス, weight : 169, cost : 303)
selected : #102 (name : ラージライス, weight : 219, cost : 454)
total price : 1106
total kcal : 1794

count : 3
y=1のindexは [531]番目.
selected : #86 (name : エビとイカのドリア, weight : 499, cost : 624)
selected : #95 (name : イタリアンハンバーグ, weight : 499, cost : 633)
total price : 998
total kcal : 1257

count : 4
y=1のindexは [597]番目.
selected : #25 (name : ポテトのグリル, weight : 199, cost : 366)
selected : #73 (name : アーリオ・オーリオ(Wサイズ), weight : 574, cost : 1120)
selected : #102 (name : ラージライス, weight : 219, cost : 454)
total price : 992
total kcal : 1940

count : 5
y=1のindexは [606]番目.
selected : #28 (name : フォッカチオ, weight : 119, cost : 2

ここではtotal priceにブレがあるものの、total kcalではtotal priceの中でtotal kcalを最大化する組み合わせを出力しているように思います。